# 1. Instalando Dependências
  Nessa seção vamos instalar a dependência DeepFace, necessário para gerar os embeddings que vamos utilizar no treino e escolha de parametros

In [ ]:
!pip install deepface
!pip install joblib
!pip install python-telegram-bot --upgrade
!pip install pyTelegramBotAPI

# 2 Importando Bibliotecas e montando sistema de arquivos

In [ ]:
import numpy as np
import os
from deepface import DeepFace
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from joblib import dump, load
from google.colab import files
from io import BytesIO
from sklearn.preprocessing import normalize
from PIL import Image
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. Criando funções de embedding

Cria embedding de uma única imagem

In [ ]:
def single_embedding(image_path):
  embedding = np.array(DeepFace.represent(image_path, 'Facenet512', enforce_detection=False, detector_backend = 'skip')[0]['embedding'])

  return embedding


Cria embedding das imagens do dataset e adiciona nos arrays de X, de dados de treino, e Y, labels dos dados de treino.

In [ ]:
def embeddings_from_folder(x,y,fpath,folder):
  for image in os.listdir(fpath):
    image_path = os.path.join(fpath,image)
    embedding = np.array(DeepFace.represent(image_path, 'Facenet512', enforce_detection=False, detector_backend = 'skip')[0]['embedding'])
    x.append(embedding)
    y.append(folder)

# 4. Criação do dataset

In [ ]:
def create_dataset(path):
    folders = []
    x = []
    y = []

    for i in os.listdir(path):
        folders.append(i)

    for folder in folders:
      print(folder)
      fpath = os.path.join(path,folder)
      embeddings_from_folder(x,y,fpath,folder)


    return np.array(x),np.array(y)


In [ ]:
x,y = create_dataset('/content/drive/MyDrive/Dataset_Wanderson_Junior')


Geffte
Fernando_Goncalves
Marcelo
Fabio_Huang
joao_lucas
Wanderson
Arthur_NM
Gabriel_Bonetti
Aninha
Jailton
Pessoas_Desconhecidas


In [ ]:
x.shape

(165, 512)

In [ ]:
y.shape

(165,)

In [ ]:
y

array(['Geffte', 'Geffte', 'Geffte', 'Geffte', 'Geffte', 'Geffte',
       'Geffte', 'Geffte', 'Geffte', 'Geffte', 'Geffte', 'Geffte',
       'Geffte', 'Geffte', 'Geffte', 'Fernando_Goncalves',
       'Fernando_Goncalves', 'Fernando_Goncalves', 'Fernando_Goncalves',
       'Fernando_Goncalves', 'Fernando_Goncalves', 'Fernando_Goncalves',
       'Fernando_Goncalves', 'Fernando_Goncalves', 'Fernando_Goncalves',
       'Fernando_Goncalves', 'Fernando_Goncalves', 'Fernando_Goncalves',
       'Fernando_Goncalves', 'Fernando_Goncalves', 'Marcelo', 'Marcelo',
       'Marcelo', 'Marcelo', 'Marcelo', 'Marcelo', 'Marcelo', 'Marcelo',
       'Marcelo', 'Marcelo', 'Marcelo', 'Marcelo', 'Marcelo', 'Marcelo',
       'Marcelo', 'Fabio_Huang', 'Fabio_Huang', 'Fabio_Huang',
       'Fabio_Huang', 'Fabio_Huang', 'Fabio_Huang', 'Fabio_Huang',
       'Fabio_Huang', 'Fabio_Huang', 'Fabio_Huang', 'Fabio_Huang',
       'Fabio_Huang', 'Fabio_Huang', 'Fabio_Huang', 'Fabio_Huang',
       'joao_lucas', 'joao_lucas

# 5. Distribuição de classes

In [ ]:
classes, qnt = np.unique(y,return_counts=True)
plt.figure()
plt.pie(qnt,labels=classes,autopct='%1.1f%%')
plt.title("Distribuição de classes")
plt.axis('equal')
plt.show()

NameError: name 'np' is not defined

In [ ]:
dist_classes = pd.DataFrame({'Classe': classes, 'Quantidade': qnt})

dist_classes

,Classe,Quantidade
0,Aninha,15
1,Arthur_NM,15
2,Fabio_Huang,15
3,Fernando_Goncalves,15
4,Gabriel_Bonetti,15
5,Geffte,15
6,Jailton,15
7,Marcelo,15
8,Pessoas_Desconhecidas,15
9,Wanderson,15


In [ ]:
np.random.seed(0)

## 6. Convertendo o vetor de labels de string para int



In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)

In [ ]:
y

array([ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8])

## 7. Validação Cruzada Estratificada e Normalização

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
X_train = normalize(X_train, norm='l2')
X_test = normalize(X_test,norm='l2')

# 8. Busca de Hiperparametros

In [ ]:
parametros = {
    'weights': ['uniform', 'distance'],
    'n_neighbors':  list(range(3, 101, 2)),
    'metric': [None,'cosine', 'euclidean', 'minkowski'],
    'p': [1, 2],
}

In [ ]:
halving = HalvingGridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=parametros,
    cv=4,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

In [ ]:
halving.fit(X_train, y_train)

n_iterations: 1
n_required_iterations: 7
n_possible_iterations: 1
min_resources_: 88
max_resources_: 132
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 784
n_resources: 88
Fitting 4 folds for each of 784 candidates, totalling 3136 fits


HalvingGridSearchCV(cv=4, estimator=KNeighborsClassifier(), n_jobs=-1,
                    param_grid={'metric': [None, 'cosine', 'euclidean',
                                           'minkowski'],
                                'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19,
                                                21, 23, 25, 27, 29, 31, 33, 35,
                                                37, 39, 41, 43, 45, 47, 49, 51,
                                                53, 55, 57, 59, 61, ...],
                                'p': [1, 2],
                                'weights': ['uniform', 'distance']},
                    scoring='f1_macro', verbose=1)

# 9. Métricas

In [ ]:
accuracy_test = halving.score(X_test, y_test)
accuracy_test

0.8662337662337662

In [ ]:
pred = halving.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       0.75      1.00      0.86         3
           2       1.00      0.67      0.80         3
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         3
           5       1.00      0.33      0.50         3
           6       0.75      1.00      0.86         3
           7       0.75      1.00      0.86         3
           8       1.00      0.67      0.80         3
           9       0.75      1.00      0.86         3
          10       1.00      1.00      1.00         3

    accuracy                           0.88        33
   macro avg       0.91      0.88      0.87        33
weighted avg       0.91      0.88      0.87        33



In [ ]:
fscore = f1_score(y_test, pred,average= 'macro')

In [ ]:
fscore

0.8662337662337662

In [ ]:
print(confusion_matrix(y_test,pred))

[[3 0 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 1 0 0 0]
 [0 0 0 3 0 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 1 0]
 [0 0 0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0 0]
 [0 1 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 0 3]]


# 10. Dumping do modelo

In [ ]:
dump(halving, 'porteiro.joblib')

['porteiro.joblib']

In [ ]:
files.download('porteiro.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 11. Criação e Polling do Bot

@PorteiroJuniorbot

In [ ]:
import telebot

BOT_TOKEN = TOKEN

bot = telebot.TeleBot(BOT_TOKEN)

@bot.message_handler(commands=['start', 'hello'])
def send_welcome(message):
    bot.reply_to(message, "Olá, sou o Porteiro Junior para que eu verifique se você é um morador do prédio, por favor me envie uma imagem sua!")

@bot.message_handler(content_types=['photo'])
def inference(message):
    id = message.photo[-1].file_id
    info = bot.get_file(id)
    download = bot.download_file(info.file_path)
    array = np.array(Image.open(BytesIO(download)))
    embedding = single_embedding(array)
    embedding = embedding.reshape(1,-1)
    embedding = normalize(embedding, norm='l2')
    print(embedding)
    inference = halving.predict(embedding)
    classe = label_encoder.inverse_transform([inference])[0]
    if classe == "Pessoas_Desconhecidas":
      bot.reply_to(message, f"Identidade desconhecida")
    else:
      bot.reply_to(message, f"Seja bem vindo Sr(a) {classe}, pode entrar!")


bot.infinity_polling()




[[-2.59405340e-03  4.40293952e-02 -4.49175201e-02  9.03309276e-02
   4.56489347e-02  6.71862643e-03 -7.10367442e-02  1.05437289e-02
  -5.80078045e-02  2.98662479e-02  1.88216401e-02  4.32748061e-02
   2.05610933e-02  2.89917792e-02  2.00624459e-02 -5.98845780e-02
   7.18440147e-03  1.06513361e-01 -3.41337186e-02  1.62811120e-02
  -2.23753323e-02 -1.32285252e-02  2.71934265e-02 -4.68005546e-02
   1.06984187e-02 -1.16136003e-02  4.41799813e-02  2.57611500e-02
   6.18003125e-03  2.33239498e-02  7.04843767e-03 -7.79774254e-03
  -5.04525719e-02  1.23978230e-03 -1.83052114e-02 -5.50756773e-02
  -2.03505104e-02 -5.53008226e-02 -9.42331143e-02  4.85348012e-02
  -1.05251847e-01  2.78274407e-02  5.71631483e-03 -2.35679722e-02
   4.55850935e-02  4.81485968e-02  3.18177687e-02 -1.27981546e-02
  -2.20134124e-02 -2.47375657e-02  1.88190611e-02 -8.45450710e-02
   3.69280611e-02 -2.34263572e-02 -6.25834437e-02 -3.76747192e-02
  -8.08868953e-02  6.76356031e-02 -1.17889262e-02  5.79913847e-04
  -1.82054

2025-06-01 00:49:27,640 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2025-06-01 00:49:27,643 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
